In [2]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary
import matplotlib.pyplot as plt

from generate_data import generate_hiv_data, generate_discrete_hiv_data
from decimal import Decimal

In [17]:
T = 1e-1
t = np.linspace(0, 100, 1001)
x0 = [1, 1, 1, 1, 1]
x0_val = [2, 1, 2, 2, 3]

x, x_dot = generate_hiv_data(t=t, x0=x0)
x_val, _ = generate_hiv_data(t=t, x0=x0_val)

xk = generate_discrete_hiv_data(t=t, x0=x0, T=T)
xk_val = generate_discrete_hiv_data(t=t, x0=x0_val, T=T)

In [27]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=2),
    # feature_library=FourierLibrary(n_frequencies=1),
    optimizer=ps.STLSQ(threshold=0.001),
    feature_names=[f'x{i+1}' for i in range(len(x[0]))],
    discrete_time=True,
    )
model.fit(x=xk)
model.print(precision=5)

(x1)[k+1] = 0.10000 1 + 0.99000 x1[k] + -0.10000 x1[k] x2[k]
(x2)[k+1] = 0.98000 x2[k] + 0.10000 x1[k] x2[k] + -0.10000 x2[k] x4[k] + -0.10000 x2[k] x5[k]
(x3)[k+1] = 0.02166 1 + -0.00019 x1[k] + -0.01503 x2[k] + 0.97890 x3[k] + -0.08547 x4[k] + -0.00228 x5[k] + -0.00015 x1[k]^2 + 0.00646 x1[k] x2[k] + 0.00039 x1[k] x3[k] + -0.00853 x1[k] x4[k] + 0.01104 x1[k] x5[k] + -0.00020 x2[k]^2 + 0.01070 x2[k] x3[k] + 0.01504 x2[k] x4[k] + -0.01411 x2[k] x5[k] + -0.00128 x3[k]^2 + 0.04711 x3[k] x4[k] + 0.02187 x3[k] x5[k] + 0.02897 x4[k]^2 + 0.00655 x4[k] x5[k] + -0.01945 x5[k]^2
(x4)[k+1] = 0.99000 x4[k] + 0.00300 x2[k] x4[k]
(x5)[k+1] = 0.99000 x5[k] + 0.00300 x2[k] x3[k]


In [11]:
for x_num in range(len(x0)):
    q = 'Biblioteka funkcji & Próg & $\Dot{x}'
    print(f'{q}_{x_num+1}$ & $E_{x_num+1}$ \\\\')
    for i, library in enumerate([FourierLibrary(n_frequencies=2), FourierLibrary(n_frequencies=1), PolynomialLibrary(degree=3), PolynomialLibrary(degree=2), PolynomialLibrary(degree=1)]):
        print(f'\\hline')
        for threshold in range(3):
            threshold = 10**(threshold-4) if i >= 2 else 100*10**(threshold-4)
            threshold = T * threshold
            name = ['Trygonometryczna (st. 2)', 'Trygonometryczna (st. 1)', 'Wielomiany (st. 3)', 'Wielomiany (st. 2)', 'Liniowa']
            model = ps.SINDy(
                feature_library=library,
                optimizer=ps.STLSQ(threshold=threshold),
                feature_names=[f'x{i+1}' for i in range(len(x0))],
                discrete_time=True)
            model.fit(x=xk)
            try:
                x_sim = model.simulate(x0=x0_val, t=1001)
                mse = ((x_sim - xk_val)**2).mean(axis=0)
                E = '%.3E' % Decimal(str(mse[x_num]))
            except:
                E = '\infty'
            coeffs = ' + '.join(['%.3E' % Decimal(str(coeff))+' '+model.get_feature_names()[i] for i, coeff in enumerate(model.coefficients()[x_num]) if abs(model.coefficients()[x_num][i]) > threshold])
            if len(coeffs.split(' + ')) > 2:
                eq = (coeffs.split(' + ')[0] + ' + ' + coeffs.split(' + ')[1] + '\dots').replace(' 1 +', ' +')
            else:
                eq = coeffs
            if len(coeffs) == 0:
                eq = '0,000'
            eq = eq.replace(' 1 +', ' +').replace('.', ',').replace('+ -', '- ').replace('sin', '\sin').replace('cos', '\cos').replace('(1 x1)', '(x_1)').replace('(1 x2)', '(x_2)').replace('x2', 'x_2').replace('(1 x3)', '(x_3)').replace('x3', 'x_3').replace('(1 x4)', '(x_4)').replace('x4', 'x_4').replace('(1 x5)', '(x_5)').replace('x5', 'x_5').replace('x1', 'x_1').replace('(1 u)', '(u)')
            for pow in range(1, 10):
                eq = eq.replace('E+00', '')
                E = E.replace('E+00', '')
                eq = eq.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
                E = E.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
            print(f"{name[i]} & {(str(threshold).replace('.', ',') + ' &').replace(',0 &', ' &')} ${eq}$ & ${E.replace('.', ',')}$ \\\\")
    print('\n\n')

Biblioteka funkcji & Próg & $\Dot{x}_1$ & $E_1$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $-4,497\cdot 10^{-1} \sin(x_1) - 1,260 \cos(x_1)\dots$ & $4,705\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $-4,033\cdot 10^{-1} \sin(x_1) - 1,255 \cos(x_1)\dots$ & $6,739\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,1 & $3,166\cdot 10^{-1} \sin(x_1) - 1,107 \cos(x_1)\dots$ & $7,540\cdot 10^{-1}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $-2,776\cdot 10^{-1} \sin(x_1) - 9,090\cdot 10^{-1} \cos(x_1)\dots$ & $6,218$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $-2,699\cdot 10^{-1} \sin(x_1) - 9,075\cdot 10^{-1} \cos(x_1)\dots$ & $6,066$ \\
Trygonometryczna (st. 1) & 0,1 & $-3,309\cdot 10^{-1} \sin(x_1) - 9,185\cdot 10^{-1} \cos(x_1)\dots$ & $9,020\cdot 10^{-1}$ \\
\hline
Wielomiany (st. 3) & 1e-05 & $1,000\cdot 10^{-1} + 9,900\cdot 10^{-1} x_1\dots$ & $7,157\cdot 10^{-3}$ \\
Wielomiany (st. 3) & 0,0001 & $1,000\cdot 10^{-1} + 9,900\cdot 10^{-1} x_1\dots$ & $7,199\cdot

In [14]:

x_sim = model.simulate(x0=x0_val, t=1001)
mse = ((x_sim - xk_val)**2).mean(axis=0)
mse

array([7.65423562e-04, 1.45524657e-04, 4.64011427e-02, 9.70198895e-07,
       3.51260927e-04])

In [ ]:
plt.plot(t, x_val[:, 0])
plt.plot(t, x_val[:, 1])
plt.plot(t, x_val[:, 2])
plt.plot(t, x_val[:, 3])
plt.plot(t, x_val[:, 4])
plt.legend(["Zdrowe CD4+", "Zainfekowane CD4+", "Prekursory LTC", "Pomocniczo-niezależne LTC", "Pomocniczo-zależne LTC"])
# plt.ylim(0, max(x[:, 0]*1.2))
plt.xlim(0, max(t))
plt.xlabel("Czas [dni]")
plt.ylabel("Stężenie")

plt.grid()
plt.show()